#### anchor5 2.5w~3w exported

In [1]:
import glob
import matplotlib.pyplot as  plt
%matplotlib inline
import os
import sys
sys.path.append('..')
import random
import pdb
from ldklib.docker_models import CTPN_Docker
import cv2
import json
import docker
import grpc
import ctpn_pb2
import ctpn_pb2_grpc
import numpy as np

CTPN_CKPT= '/trnet_fs/ldktemp/qhctpn35/checkpoints_address/'
CTPN_CKPT= '/home/lidingke/Weights/label_fd/fd0035'


class CTPN_Docker_Temp(CTPN_Docker):

    def __init__(self):
        CTPN_Docker.__init__(self,docker_client = 'unix:///var/run/docker.sock',
                                         host_port ='localhost:50052')
        self.run_args = []
        self.run_kwargs = {'image' : "trnet/ctpn:1.0.1",
                    'command' : "python rpc/server.py",
                    'runtime':'nvidia',
                    'environment' : ["CUDA_VISIBLE_DEVICES=0"],
                    'ports' : {'50051/tcp':'50052'},
                    'volumes' :{CTPN_CKPT:\
                              {'bind': '/ctpn_deploy/ckpt', 'mode': 'ro'},
                             '/trnet_fs/ldktemp/qhctpn35/text_connect_cfg_dk.py':\
                              {'bind': '/ctpn_deploy/lib/text_connector/text_connect_cfg.py', 'mode': 'ro'}},
                    'detach':True,
                    'auto_remove' : True}

if __name__ == '__main__':
    results = []
    with CTPN_Docker_Temp() as sess:
        with open('/trnet_fs/LDKDATA/txts/ALLFD0035.txt', 'r') as f:
            lines = f.readlines()
            splits = [e.strip().split('\t') for e in lines if e.strip()!='']
#             random.shuffle(splits)
            for id_,lo,fd,label,dir_ in splits[35000:40000]:
                if label.find('?') > 0:
                    continue
                img = cv2.imread(dir_)
                
                if isinstance(img,np.ndarray) :
                    im_cp = img.copy()

                    rects = sess.run(img)
                    _ = (id_,fd,label,dir_,rects)
                    results.append(_)
    
                for r in rects:
                    xmin,ymin,xmax,ymax = r
                    cv2.rectangle(im_cp, (xmin,ymin), (xmax,ymax), (0, 0, 255), 2)
#                 print('result:',r,'type',type(r))
                if rects and int(id_)%1000 == 0:
                    plt.imshow(im_cp)
                    plt.show()
                
    print(len(results))

KeyboardInterrupt: 

#### anchor7 30000:35000 exported
#### anchor8 35000:40000 exported

In [3]:
!ls /home/lidingke/DATA/ALLFD0035/

anchor0  anchor1_byLei	anchor4      anchor5	  anchor6
anchor1  anchor2	anchor4.zip  anchor5.zip


In [2]:
DATA_FILE='/trnet_fs/LDKDATA/ALLFD0035/anchor8'
os.path.join(DATA_FILE,'JPEGImages','%s.jpg'%'1')

'/trnet_fs/LDKDATA/ALLFD0035/anchor8/JPEGImages/1.jpg'

In [3]:
%env DATA_FILE=/trnet_fs/LDKDATA/ALLFD0035/anchor8

env: DATA_FILE=/trnet_fs/LDKDATA/ALLFD0035/anchor8


In [5]:
%%bash

mkdir $DATA_FILE
mkdir $DATA_FILE/JPEGImages
mkdir $DATA_FILE/Annotations
echo $DATA_FILE/JPEGImages
echo $DATA_FILE/Annotations

/trnet_fs/LDKDATA/ALLFD0035/anchor8/JPEGImages
/trnet_fs/LDKDATA/ALLFD0035/anchor8/Annotations


In [6]:
import sys
sys.path.append('..')
from ldklib.voc_xml import voc_xml_save,voc_xml_get

def fun(result):
    id_,fd,label,dir_,rects = result
    img = cv2.imread(dir_)
    informs = [(label,r) for r in rects]  
    impath = os.path.join(DATA_FILE,'JPEGImages','%s.jpg'%id_)
    xmlpath = os.path.join(DATA_FILE,'Annotations','%s.xml'%id_)
    cv2.imwrite(impath,img)
    voc_xml_save(id_,informs,img.shape,xmlpath)

from multiprocessing.dummy import Pool

p = Pool(10)
len(p.map(fun,results))

4856

In [7]:
!ls $DATA_FILE/JPEGImages | wc
!ls $DATA_FILE/Annotations/ | wc
!ls $DATA_FILE/Annotations/ | head -n 10

   4856    4856   58272
   4856    4856   58272
3581343.xml
3581433.xml
3581462.xml
3581472.xml
3581545.xml
3581634.xml
3581799.xml
3582006.xml
3582080.xml
3582255.xml


In [27]:
with open('/trnet_fs/LDKDATA/txts/ALLFD0014.txt','r') as f:
    line = f.readlines()
import cv2
lines = [l.strip().split('\t') for l in line]
writes = []
for li in lines:
    if li[0]:
        _ = "{}:{}\n".format(li[-1].strip(),li[3])
#         print("li",_,li[0],li[-1].strip())
        im = cv2.imread(li[-1].strip())
        if isinstance(im,np.ndarray):
            writes.append(_)
with open('/trnet_fs/LDKDATA/txts/ALLFD0014_forqh.txt','w') as f:
    f.writelines(writes)

In [28]:
!head -n 10 /trnet_fs/LDKDATA/txts/ALLFD0014_forqh.txt
!cat /trnet_fs/LDKDATA/txts/ALLFD0014_forqh.txt | wc

/trnet_fs/data0410/LO0151/FD0014/48518.JPG:13589858832
/trnet_fs/data0410/LO0151/FD0014/48533.JPG:15563890518
/trnet_fs/data0410/LO0151/FD0014/48548.JPG:13620614339
/trnet_fs/data0410/LO0151/FD0014/48563.JPG:19981995979
/trnet_fs/data0410/LO0151/FD0014/48578.JPG:17302109562
/trnet_fs/data0410/LO0145/FD0014/48593.JPG:17785110916
/trnet_fs/data0410/LO0145/FD0014/48608.JPG:13752117711
/trnet_fs/data0410/LO0149/FD0014/48619.JPG:13519889887
/trnet_fs/data0410/LO0151/FD0014/48634.JPG:15021155563
/trnet_fs/data0410/LO0151/FD0014/48649.JPG:18201853197
  62377   62378 3577999
